# ETL Pipeline for Graph Database Construction

This notebook implements the **ETL process** for preparing a property graph database
for Australian road safety and fatal crash analysis.

It transforms the original relational-style dataset into **node and relationship CSV files**
compatible with **Neo4j bulk import**, following the graph schema defined in the project report.

## Purpose of This Notebook

The ETL pipeline performs the following tasks:

- Cleans and normalizes the original crash dataset
- Constructs node tables (Crash, Person, Vehicle, Location, Road, Time)
- Constructs relationship tables linking crashes to related entities
- Generates stable IDs for nodes and relationships
- Outputs CSV files for direct Neo4j import

The resulting files are stored in the `data/csv_full_datasets/` directory.

## Academic Context

This work was originally developed as **Project 2** for  
**CITS5504 – Data Warehousing**, University of Western Australia.

Author: Boya Zhang  
Date: May 2025

## 1. Setup

**Importing the libraries**

In [1]:
# Import libraries for data manipulation and analysis
import pandas as pd

**Loading the datasets**

In [2]:
# Load dataset
crash_df = pd.read_csv('Project2_Dataset_Corrected.csv')
crash_df.head()

,ID,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Number Fatalities,Bus Involvement,...,Age,National Remoteness Areas,SA4 Name 2021,National LGA Name 2024,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,1,20241115,NSW,12,2024,Friday,4:00,Single,1,No,...,74,Inner Regional Australia,Riverina,Wagga Wagga,Arterial Road,Yes,No,65_to_74,Weekday,Night
1,2,20241125,NSW,12,2024,Friday,6:15,Single,1,No,...,19,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury,Local Road,No,No,17_to_25,Weekday,Day
2,3,20246013,TAS,12,2024,Friday,9:43,Single,1,No,...,33,Inner Regional Australia,Launceston and North East,Northern Midlands,Local Road,Yes,No,26_to_39,Weekday,Day
3,4,20241002,NSW,12,2024,Friday,10:35,Single,1,No,...,32,Outer Regional Australia,New England and North West,Armidale,National or State Highway,No,No,26_to_39,Weekday,Day
4,5,20243185,QLD,12,2024,Friday,13:00,Single,1,No,...,61,Inner Regional Australia,Toowoomba,Lockyer Valley,National or State Highway,No,No,40_to_64,Weekday,Day


In [3]:
print(f"Dataset shape: {crash_df.shape}")

Dataset shape: (10490, 25)


## 2. Dataset Overview

In this section, I examine the original dataset to understand its structure and assess data quality. This includes checking the column names, data types, and the presence of missing values.

In [4]:
crash_df.columns

Index(['ID', 'Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time',
       'Crash Type', 'Number Fatalities', 'Bus Involvement',
       'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement',
       'Speed Limit', 'Road User', 'Gender', 'Age',
       'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2024',
       'National Road Type', 'Christmas Period', 'Easter Period', 'Age Group',
       'Day of week', 'Time of day'],
      dtype='object')

Check for Missing Values and Data Types

In [5]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10490 entries, 0 to 10489
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   ID                             10490 non-null  int64 
 1   Crash ID                       10490 non-null  int64 
 2   State                          10490 non-null  object
 3   Month                          10490 non-null  int64 
 4   Year                           10490 non-null  int64 
 5   Dayweek                        10490 non-null  object
 6   Time                           10490 non-null  object
 7   Crash Type                     10490 non-null  object
 8   Number Fatalities              10490 non-null  int64 
 9   Bus Involvement                10490 non-null  object
 10  Heavy Rigid Truck Involvement  10490 non-null  object
 11  Articulated Truck Involvement  10490 non-null  object
 12  Speed Limit                    10490 non-null  int64 
 13  R

All columns contain 10,490 non-null values, meaning there is no missing data in the dataset.
The data types are appropriate for transformation: numeric columns use int64, and categorical fields use object. Therefore, no imputation or row removal is needed.

These columns will later be mapped to graph nodes, properties, and relationships based on the data model defined in Section 2.


## 3. Create Node CSVs

In this section, I extract and transform the data into separate node and relationship files for Neo4j.

For each node:
1. Select the columns relevant to that entity.
2. Drop duplicate rows to ensure each node is unique.
3. Create a unique identifier column (ID) if the dataset does not already contain one.

These steps ensure that each node file has one row per entity and can be linked via relationships in Neo4j.

### Crash Node

In [6]:
# Create Crash node file
# Extract relevant columns and remove duplicate crash records
crash_node_table = crash_df[['Crash ID', 'Crash Type', 'Number Fatalities']].drop_duplicates()

# Export the result to a CSV file
crash_node_table.to_csv('csv_full_datasets/crash_node.csv', index=False)

# Print the number of unique crash nodes
print(f"Crash nodes: {crash_node_table.shape}")

Crash nodes: (9683, 3)


### Vehicle Node

In [7]:
# Create Vehicle node file
# Extract vehicle-related fields and drop duplicate combinations
vehicle_node_table = crash_df[['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']].drop_duplicates()

# Add a unique vehicleID for each row (needed as Neo4j node ID)
vehicle_node_table.insert(0, 'vehicleID', range(1, 1 + len(vehicle_node_table)))

# Export to CSV
vehicle_node_table.to_csv('csv_full_datasets/vehicle_node.csv', index=False)

# Print the number of unique vehicle nodes
print(f"Vehicle nodes: {vehicle_node_table.shape}")

Vehicle nodes: (7, 4)


### Killed_Person Node

In [8]:
# Create Killed_Person node file
# Extract individual-level fields related to the person killed in the crash
killed_person_node_table = crash_df[['ID', 'Road User', 'Gender', 'Age', 'Age Group']]

# Rename ID column to personID for clarity in Neo4j
killed_person_node_table = killed_person_node_table.rename(columns={'ID': 'personID'})

# Export to CSV
killed_person_node_table.to_csv('csv_full_datasets/killed_person_node.csv', index=False)

# Print the number of person records
print(f"Killed_Person nodes: {killed_person_node_table.shape}")

Killed_Person nodes: (10490, 5)


### Time_Info Node

In [9]:
# Create TimeInfo node file
# Standardise the 'Time' column to HH:mm format (e.g., 4:00 → 04:00)
crash_df['Time'] = crash_df['Time'].apply(
    lambda t: f"{t.split(':')[0].zfill(2)}:{t.split(':')[1].zfill(2)}" if pd.notnull(t) and ':' in t else t
)

# Extract all time-related fields and remove duplicates
timeinfo_node_table = crash_df[['Time', 'Time of day', 'Day of week', 'Month', 'Year', 'Dayweek', 
                               'Christmas Period', 'Easter Period']].drop_duplicates()

# Add unique ID for time info
timeinfo_node_table.insert(0, 'timeID', range(1, 1 + len(timeinfo_node_table)))

# Export to CSV
timeinfo_node_table.to_csv('csv_full_datasets/timeinfo_node.csv', index=False)

# Print the number of time nodes
print(f"TimeInfo nodes: {timeinfo_node_table.shape}")

TimeInfo nodes: (9424, 9)


## Road Node

In [10]:
# Create Road node file
# Extract road-related attributes and remove duplicates
road_node_table = crash_df[['Speed Limit', 'National Road Type']].drop_duplicates()

# Add road ID
road_node_table.insert(0, 'roadID', range(1, 1 + len(road_node_table)))

# Export to CSV
road_node_table.to_csv('csv_full_datasets/road_node.csv', index=False)

# Print number of road nodes
print(f"Road nodes: {road_node_table.shape}")

Road nodes: (80, 3)


### Location Node

In [11]:
# Create Location node file
# Extract geographic fields and remove duplicates
location_node_table = crash_df[['National LGA Name 2024', 'SA4 Name 2021', 'State', 
                               'National Remoteness Areas']].drop_duplicates()

# Standardize column names for consistency
location_node_table = location_node_table.rename(columns={
    'National LGA Name 2024': 'lgaName',
    'SA4 Name 2021': 'sa4',
    'State': 'state',
    'National Remoteness Areas': 'nationalRemotenessAreas'
})

# Add a unique location ID
location_node_table.insert(0, 'locationID', range(1, 1 + len(location_node_table)))

# Export to CSV
location_node_table.to_csv('csv_full_datasets/location_node.csv', index=False)
print(f"Location nodes: {location_node_table.shape}")

Location nodes: (786, 5)


## 4. Create Relationship CSVs

### Relationship 1: Vehicle -[INVOLVED_IN]-> Crash
This relationship connects each crash to the type of vehicle involved.

In [12]:
# Create INVOLVED_IN relationship file between Vehicle and Crash
# Extract crash-vehicle combinations and remove duplicates
rel_involved_in_table = crash_df[['Crash ID', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 
                                'Articulated Truck Involvement']].drop_duplicates()

# Merge to get vehicleID from vehicle_node_table
rel_involved_in_table = rel_involved_in_table.merge(
    vehicle_node_table,
    on=['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement'],
    how='left'
)

# Keep only IDs needed for relationship file
rel_involved_in_table = rel_involved_in_table[['vehicleID', 'Crash ID']].rename(columns={'Crash ID': 'crashID'})

# Export to CSV
rel_involved_in_table.to_csv('csv_full_datasets/rel_involved_in.csv', index=False)

# Show total number of relationships
print(f"INVOLVED_IN relationships: {len(rel_involved_in_table)}")

INVOLVED_IN relationships: 9683


### Relationship 2: KILLED_IN – Killed_Person → Crash

In [13]:
# Create KILLED_IN relationship between Killed_Person and Crash
# Each person is linked to exactly one crash
rel_killed_in_table = crash_df[['ID', 'Crash ID']].rename(columns={'ID': 'personID', 'Crash ID': 'crashID'})

# Export to CSV
rel_killed_in_table.to_csv('csv_full_datasets/rel_killed_in.csv', index=False)
print(f"KILLED_IN relationships: {len(rel_killed_in_table)}")

KILLED_IN relationships: 10490


### Relationship 3: HAPPENED_DURING – Crash → TimeInfo

In [14]:
# Create HAPPENED_DURING relationship between Crash and TimeInfo
# Join based on all time-related fields to get corresponding timeID
rel_happened_during_table = crash_df[['Crash ID', 'Time', 'Time of day', 'Day of week', 'Month', 'Year', 
                               'Dayweek', 'Christmas Period', 'Easter Period']].drop_duplicates()

rel_happened_during_table = rel_happened_during_table.merge(
    timeinfo_node_table,
    on=['Time', 'Time of day', 'Day of week', 'Month', 'Year', 'Dayweek', 
        'Christmas Period', 'Easter Period'],
    how='left'
)

rel_happened_during_table = rel_happened_during_table[['Crash ID', 'timeID']].rename(columns={'Crash ID': 'crashID'})
rel_happened_during_table.to_csv('csv_full_datasets/rel_happened_during.csv', index=False)
print(f"HAPPENED_DURING relationships: {len(rel_happened_during_table)}")

HAPPENED_DURING relationships: 9683


### Relationship 4: HAPPENED_ON – Crash → Road

In [15]:
# Create HAPPENED_ON relationship between Crash and Road
# Join crash info with road node info to get roadID
rel_happened_on_table = crash_df[['Crash ID', 'Speed Limit', 'National Road Type']].drop_duplicates()

rel_happened_on_table = rel_happened_on_table.merge(
    road_node_table,
    on=['Speed Limit', 'National Road Type'],
    how='left'
)

rel_happened_on_table = rel_happened_on_table[['Crash ID', 'roadID']].rename(columns={'Crash ID': 'crashID'})
rel_happened_on_table.to_csv('csv_full_datasets/rel_happened_on.csv', index=False)
print(f"HAPPENED_ON relationships: {len(rel_happened_on_table)}")

HAPPENED_ON relationships: 9683


### Relationship 5: LOCATED_IN – Crash → Location

In [16]:
# Create LOCATED_IN relationship between Crash and Location
# Join crash location info with location node to get locationID
rel_located_in_table = crash_df[['Crash ID', 'National LGA Name 2024', 'SA4 Name 2021', 'State', 
                               'National Remoteness Areas']].drop_duplicates()

rel_located_in_table = rel_located_in_table.merge(
    location_node_table,
    left_on=['National LGA Name 2024', 'SA4 Name 2021', 'State', 'National Remoteness Areas'],
    right_on=['lgaName', 'sa4', 'state', 'nationalRemotenessAreas'],
    how='left'
)

rel_located_in_table = rel_located_in_table[['Crash ID', 'locationID']].rename(columns={'Crash ID': 'crashID'})
rel_located_in_table.to_csv('csv_full_datasets/rel_located_in.csv', index=False)
print(f"LOCATED_IN relationships: {len(rel_located_in_table)}")

LOCATED_IN relationships: 9683
